In [1]:
from datetime import timedelta, date
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import pyspark.sql.functions as func
import time

sqlContext.setConf('spark.sql.shuffle.partitions', '6')

# Function for Changing Data Type

In [2]:
##Fucntion for chaging the datatype
def data_type_change_int(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(IntegerType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

def data_type_change_float(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(DoubleType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

# UDF for Bucketing Fields from HOTEL_DIM Table

In [3]:
#Star Rating Buckets
def srch_region_star_rating(star_rating):
    if star_rating == '2.0':
        return '2'
    elif star_rating == '2.5':
        return '2.5'
    elif star_rating == '3.0':
        return '3'
    elif star_rating == '3.5':
        return '3.5'
    elif star_rating == '4.0':
        return '4'
    elif star_rating == '1.0' or star_rating == '1.5':
        return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0':
        return '4.5-5'
    else:
        return '0'
    
#City Buckets
def srch_region_city(city,model_name):
	if model_name == 'CANADA':
		if city in ('TORONTO' , 'NIAGARA FALLS' , 'MONTREAL' , 'VANCOUVER' , 'QUEBEC' , 'BANFF' , 'OTTAWA' , 'EDMONTON' , 'VICTORIA','CALGARY' , 'WHISTLER'):
			return city
		elif city in ('NIAGARA-ON-THE-LAKE','CANMORE','JASPER','KELOWNA','HALIFAX','MISSISSAUGA','LAKELOUISE','COLLINGWOOD','VERNON','PENTICTON','GATINEAU','DORVAL','MARKHAM'):
			return'G12'
		elif city in ('MONT-TREMBLANT','TOFINO','BLUEMOUNTAINS','OSOYOOS','UCLUELET','SAINT-SAUVEUR','WESTKELOWNA','LAMALBAIE','SAINTE-ADELE'):
			return'G13'
		elif city in ('WINNIPEG','LONDON','RICHMOND','BARRIE','REVELSTOKE','VAUGHAN','ORILLIA','BURNABY','SURREY'):
			return 'G14'
		else:
			return 'others'
	else:
		return 'NA'
    

In [4]:
#udf for hotel type 
def srch_region_Hotel_type(Hotel_type,model_name):
	if model_name == 'CANADA':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' , 'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL', 'INN'):
			return 'Family_Premium'
		else:
			return 'Others'
	
	elif model_name == 'US':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,  'INN' , 'APARTMENT', 'VILLA', 'PRIVATE VACATION HOME'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Mexico':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE') :
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'APART-HOTEL') :
			return Hotel_type
		elif Hotel_type in ('GUEST HOUSE' ,'BED & BREAKFAST', 'HOSTEL/BACKPACKER ACCOMMODATION', 'MOTEL' , 'HOSTAL (BUDGET HOTEL)'):
			return 'Budget_hotel'
		elif Hotel_type in ('APARTMENT','RESIDENCE','TOWNHOUSE','INN'):
			return 'Budget_hotel'
		else:
			return 'others'
	
	elif model_name=="North America":
		if Hotel_type in ('HOTEL' , 'MOTEL') :
			return Hotel_type
		elif Hotel_type in ('LODGE' ,  'BED & BREAKFAST' ,  'COTTAGE' ,  'GUEST HOUSE' ,  'CABIN'):
			return 'Budget_hotel'
		elif Hotel_type in ( 'APARTMENT' ,  'HOTEL RESORT' ,  'CONDO' ,  'INN' ,  'APART-HOTEL' ,  'CARAVAN PARK'):
			return 'Family_Premium'
		else:
			return 'others'
	else:
		return 'NOT APPLICABLE'

In [5]:
##udf for market bucket
def srch_region_market(market,model_name):
	if model_name == 'CANADA':
		if market in  ('TORONTO, ON, CAN' , 'NIAGARA FALLS, ON, CAN' , 'MONTREAL, QC, CAN' , 'BANFF AREA, AB, CAN' ,'VANCOUVER, BC, CAN' ,'QUEBEC CITY, QC, CAN' , 'EDMONTON, AB, CAN' , 'OTTAWA, ON, CAN' , 'OKANAGAN VALLEY, BC, CAN'):
			return market
		elif market in ('VICTORIA, BC, CAN' ,'CALGARY, AB, CAN' ,'JASPER AREA, AB, CAN'):
			return 'G10'
		elif market in ('WHISTLER, BC, CAN' ,'NIAGARA-ON-THE-LAKE, ON, CAN' ,'MONT TREMBLANT, QC, CAN' , 'UCLUELET, BC, CAN','TOFINO - UCLUELET, BC, CAN'):
			return 'G11'
		elif market in ('COLLINGWOOD, ON, CAN' , 'BC INTERIOR SKI, BC, CAN' ,'HALIFAX, NS, CAN','WINNIPEG, MB, CAN' ,'LONDON, ON, CAN' , 'KAMLOOPS, BC, CAN'):
			return 'G12'
		else:
			return 'others'
	
	elif model_name == 'US':
		return 'NA'
		
	elif model_name == 'Mexico':
		if market in  ('MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM' , 'MEXICO - CANCUN & ISLA MUJERES' ,'MEXICO - PUERTO VALLARTA, NUEVO VALLARTA & RIVIERA NAYARIT' , 'MEXICO - LOS CABOS' , 'CENTRAL AMERICA - COSTA RICA - GUANACASTE' ,'CENTRAL AMERICA - BELIZE' , 'CENTRAL AMERICA - COSTA RICA - PUNTARENAS' , 'CENTRAL AMERICA - COSTA RICA - OTHER' , 'MEXICO - COZUMEL''MEXICO - HUATULCO' , 'MEXICO - IXTAPA & ZIHUATANEJO' , 'CENTRAL AMERICA - HONDURAS' , 'CENTRAL AMERICA - PANAMA - OTHER') :
			return market
		elif market in ( 'CENTRAL AMERICA - COSTA RICA - SAN JOSE' , 'CENTRAL AMERICA - PANAMA - PANAMA CITY' ,'MEXICO - MEXICO CITY' ,'MEXICO - MEXICO CITY'						,'CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR','MEXICO - ACAPULCO'):
			return 'hclt'
		elif market in ('CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR',                        'MEXICO - ACAPULCO'):
			return 'lclt'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if market in ('PUNTA CANA, DOMINICAN REPUBLIC' , 'TURKS AND CAICOS' , 'MONTEGO BAY, JAMAICA' , 'ST. LUCIA' ,'OCHO RIOS, JAMAICA' ,                      'NEGRIL, JAMAICA' , 'BARBADOS'):
			return market
		elif market in ('ARUBA' , 'PUERTO PLATA, DOMINICAN REPUBLIC' , 'ANTIGUA AND BARBUDA'):
			return 'G8'
		elif market in ('NASSAU, BAHAMAS', 'DOMINICAN REPUBLIC ALL OTHER' , 'BERMUDA','BAHAMAS, OUT ISLANDS'):
			return 'G9'
		elif market in ('SAN JUAN, PUERTO RICO' ,'PARADISE ISLAND, BAHAMAS' ,'CAYMAN ISLANDS','SINT MAARTEN, DUTCH' ,'CURACAO','ST. MARTIN, FRENCH'):
			return 'G10'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		return 'NA'
	
	elif model_name=="North America":
		if  market in (  'GRAVENHURST - BRACEBRIDGE - HUNTSVILLE, ON, CAN','ONTARIO WEST, ON, CAN','ABBOTSFORD - CHILLIWACK, BC, CAN',                       'MAINE - COAST','ONTARIO EAST, ON, CAN','VANCOUVER ISLAND, BC, CAN','BURLINGTON, VT','CHARLOTTETOWN, PEI, CAN'): 
			return 'G1'
		elif market in ('ALBERTA EAST, AB, CAN','REGINA, SK, CAN','KINGSTON, ON, CAN','SASKATOON, SK, CAN','MONCTON, NB, CAN',                        'TRI-CITIES (KW-GUELPH-CAMBRIDGE), ON, CAN'): 
			return 'G2'
		elif market in ('WINDSOR, ON, CAN','RED DEER, AB, CAN','BRITISH COLUMBIA SOUTH, BC, CAN','GRAND FORKS, ND','SUDBURY, ON, CAN',                        'ONTARIO NORTH, ON, CAN','BANGOR, ME','HAMILTON - BRANTFORD, ON, CAN','BELLEVILLE - TRENTON - COBOURG, ON, CAN','FARGO, ND'): 
			return 'G3'
		elif market in ('LAKE PLACID, NY','PARKSVILLE, VANCOUVER ISLAND, BC, CAN','LAKE GEORGE, NY','OGUNQUIT, ME','EVERETT, WA',                        'COASTAL, BC, CAN','GROVE CITY, PA','BELLINGHAM, WA','NORTH CONWAY, NH','NEWFOUNDLAND OTHER, NL, CAN',                        'NOVA SCOTIA SOUTH, NS, CAN','PORTSMOUTH, NH'): 
			return 'G4'
		elif market in ('LETHBRIDGE, AB, CAN','FREDERICTON, NB, CAN','BRANDON, MB, CAN','ERIE, PA','NANAIMO, VANCOUVER ISLAND, BC, CAN',                        'SASKATCHEWAN SOUTH, SK, CAN','PRINCE GEORGE - QUESNEL, BC, CAN','MEDICINE HAT, AB, CAN','GRANDE PRAIRIE, AB, CAN',                        'ALBERTA NORTH, AB, CAN','DURHAM, ON, CAN','BRITISH COLUMBIA CENTRAL, BC, CAN','ONTARIO SOUTHWEST, ON, CAN',                        'SAULT STE. MARIE, ON, CAN','THUNDER BAY, ON, CAN','SAINT JOHN, NB, CAN','GREAT FALLS, MT','MINOT, ND (AREA)',                        'BRITISH COLUMBIA NORTH, BC, CAN','NORTH BAY, ON, CAN','PETERBOROUGH, ON, CAN','CAPE BRETON ISLAND, NS, CAN','ALBERTA WEST, AB, CAN',                        'HUDSON VALLEY - POUGHKEEPSIE, NY','FORT MCMURRAY, AB, CAN','NOVA SCOTIA NORTH, NS, CAN','WATERTOWN, NY','EDMUNSTON, NB, CAN',                        'OLYMPIC NATIONAL PARK AREA, WA','CEDAR CITY - BRYCE CANYON NATIONAL PARK, UT','SUMMERSIDE, PEI, CAN','SALEM, OR (AREA)','BUTTE - HELENA, MT'                        ,'BECKLEY, WV'): 
			return 'G5'
		elif market in ('SAGINAW, MI','WHITE MOUNTAINS, NH','FINGER LAKES NEW YORK','NEW YORK - WEST','NEW BRUNSWICK NORTH, NB, CAN',                        'SASKATCHEWAN NORTH, SK, CAN','MANITOBA SOUTH, MB, CAN','BAR HARBOR, ME','WASHINGTON NORTHWEST','VERMONT CENTRAL','ALBERTA SOUTH, AB, CAN',                        'WENATCHEE-LEAVENWORTH,WA','GANANOQUE, ON, CAN','PLATTSBURGH, (NY)','STRAITS OF MACKINAC, MI','SANDUSKY, OH','MANCHESTER, NH','ITHACA, NY',                        'MINNESOTA NORTHWEST','MISSOULA, MT (AREA)','YELLOWKNIFE, NT, CAN','PEI OTHER, PEI, CAN','BILLINGS, MT (AREA)','RAPID CITY - MOUNT RUSHMORE, SD',                        'ANDOVER, MA','YUMA, AZ','PORT HURON, MI','LANCASTER, PA','FORT LEE - PARAMUS, NJ','LEXINGTON, KY','MANITOBA NORTH, MB, CAN','ABERDEEN - OCEAN SHORES, WA',                        'CANADA TERRITORIES, YT, CAN','KNOXVILLE, TN','BURLINGTON - MOUNT VERNON, WA','FREDERICKSBURG, VA','ARIZONA SOUTHEAST',                        'WINDHAM COUNTY, VT','KENAI PENINSULA, AK','COLUMBIA, SC','CALIFORNIA DESERTS SOUTH'): 
			return 'G6'
		elif  market in ('ROME-UTICA, NY','SANDPOINT, ID (AREA)','MOAB - GREEN RIVER, UT','WHITEHORSE, YT, CAN','NEW YORK - NORTHEAST',                         'SARNIA, ON, CAN','AUGUSTA, ME','LAKE POWELL - GLEN CANYON NATIONAL PARK, AZ','NEW YORK - SOUTH','BISMARK, ND',                         'ALABAMA NORTH','ASHLAND - MEDFORD, OR','WASHINGTON NORTHEAST','THE BERKSHIRES, MA','EUGENE, OR (AREA)',                         'GRAND RAPIDS, MI','WASHINGTON SOUTHWEST','LAKE HAVASU CITY, AZ','SARATOGA, NY','WYTHEVILLE - VA',                         'ARCATA - EUREKA, CA','SOUTH CAROLINA EAST','CHATTANOOGA, TN','TOLEDO, OH','IDAHO FALLS, ID','WEST VIRGINIA SOUTH',                         'SPRINGFIELD, MA','OLYMPIA, WA','GETTYSBURG - PA'): 
			return 'G7'
		else:return 'Others'
	else:return 'NOT APPLICABLE'

In [6]:
## UDF for Model & Region:
def srch_model_name(region):
	if region == 'CANADA':
		return region
		
	elif region in ('FLORIDA (USA)','NEW YORK CITY','CALIFORNIA','GAMING (USA)','NORTHEAST (USA)','MOUNTAIN (USA)',                    'MIDWEST (USA)','CENTRAL (USA)','SF PACNW','ORLANDO','HAWAII'):
		return 'US'
		
	elif region in ("MEXICO - DOMESTIC REGION",'MEXICO - RESORTS REGION','CENTRAL AMERICA REGION'):
		return 'Mexico'
		
	elif region=="CARIBBEAN":
		return 'CARIBBEAN'
    
	elif region=="NORTH AMERICA REGIONAL TERRITORIES":
		return 'North America'
		
	else :
		return "Other Region"
    
def srch_region_bucket(region,model_name):
	if model_name == 'Other Region':
		if region in ('SPAIN & PORTUGAL' , 'ROME & ITALY RESORTS' , 'MID-ATLANTIC' , 'GREECE & TURKEY' , 'OCEANIA' , 'ITALY NORTH' ,                      'THAILAND' , 'MIDDLE EAST AND INDIAN OCEAN'):
			return region
		elif region in ('PARIS' , 'LONDON' , 'ENGLAND'):
			return 'G9'
		elif region in ('EUROPE REGIONAL TERRITORIES' , 'UK & IRELAND'):
			return 'G10'
		elif region in ('SOUTH AMERICA' , 'AFRICA AND EAST MED'):
			return 'G11'
		elif region in ('EASTERN EUROPE' , 'GERMANY, AUSTRIA & SWITZERLAND' , 'BENELUX' , 'FRANCE' , 'NORDIC'):
			return 'G12'
		elif region in ('JAPAN & MICRONESIA' , 'HONG KONG & MACAU' , 'SOUTHEAST ASIA EMERGING' , 'MAINLAND CHINA'):
			return 'G13'
		else:
			return 'others'
	else :
		return 'NA'

In [7]:
## Initializing all the UDFs

udfsrch_region_star_rating=udf(srch_region_star_rating, StringType())
udfsrch_region_city=udf(srch_region_city, StringType())
udfsrch_region_hotel_type=udf(srch_region_Hotel_type, StringType())
udfsrch_region_market=udf(srch_region_market, StringType())
udfsrch_model_name= udf(srch_model_name,StringType())
udfsrch_region_bucket = udf (srch_region_bucket,StringType())

# Importing Hotel Dimension Table

In [80]:
## Loading Hotel Dim table:
path_hotel_dim = 's3n://ewe-core-meta-prod/CORE/HOTEL_DIM/*'
hotel_dim = (sqlCtx.read.format("com.databricks.spark.csv")
            .options(header = False, inferSchema = True, delimiter = "\t", nullValue= '\\N')
            .load(path_hotel_dim)
            .distinct()
            .filter("C12 = 'BOOKABLE' or C12 ='NOT BOOKABLE' or C12 ='NOT APPLICABLE'")
            .select("C0","C7","C25","C26","C4","C13","C3")
            .withColumnRenamed("C0","HOTEL_ID")
            .withColumnRenamed("C7" , "city")
            .withColumnRenamed("C25","region")
            .withColumnRenamed("C26","market")
            .withColumnRenamed("C4","brand")
            .withColumnRenamed("C13","star_rating")
            .withColumnRenamed("C3","hotel_type")
            .filter("HOTEL_ID!='None'")
            )               


In [81]:
## Finalizing Hotel Dim Table
hotel_dim_with_bkts = (hotel_dim.withColumn("MODEL_NAME", udfsrch_model_name("region"))
                        .withColumn("STAR_RATING_BUCKET", udfsrch_region_star_rating("star_rating"))
                        .withColumn("HOTEL_TYPE_BUCKET", udfsrch_region_hotel_type("hotel_type","MODEL_NAME"))
                        .withColumn("MARKET_BUCKET",udfsrch_region_market("market","MODEL_NAME"))
                        .withColumn("REGION_BUCKET", udfsrch_region_bucket("region","MODEL_NAME"))
                        .withColumn("CITY_BUCKET", udfsrch_region_city("city","MODEL_NAME")))

hotel_dim_final = (hotel_dim_with_bkts
                   .drop('city')
                   .drop('market')
                   .drop('brand')
                   .drop('star_rating')
                   .drop('hotel_type')
                   .repartition(1)
                   .cache())

# UDF for  Bucketing LOS and BW from Hotel Click Book Agg Table

In [82]:
##los bucket for search los
def searchToCategory(srch_los,):
    if srch_los in (0,1,2,3,4,5,6,7):
        return str(srch_los)
    elif srch_los >= 8 and srch_los <= 14:
        return '8-14'
    else:
        return '>14'

##bw bucket for srch bw 
def searchbucketToCategory(SRCH_WINDOW):
    if SRCH_WINDOW <= 0:
        return '<=0'
    elif SRCH_WINDOW == 1:
        return '1'
    elif SRCH_WINDOW == 2:
        return '2'
    elif SRCH_WINDOW >= 3 and SRCH_WINDOW <= 4:
        return '3-4'
    elif SRCH_WINDOW >= 5 and SRCH_WINDOW <= 7:
        return '5-7'
    elif SRCH_WINDOW >= 8 and SRCH_WINDOW <= 14:
        return '8-14'
    elif SRCH_WINDOW >= 15 and SRCH_WINDOW <= 20:
        return '15-20'
    elif SRCH_WINDOW >= 21 and SRCH_WINDOW <= 30:
        return '21-30'
    elif SRCH_WINDOW >= 31 and SRCH_WINDOW <= 40:
        return '31-40'
    elif SRCH_WINDOW >= 41 and SRCH_WINDOW <= 60:
        return '41-60'
    elif SRCH_WINDOW >= 61 and SRCH_WINDOW <= 90:
        return '61-90'
    elif SRCH_WINDOW >= 91 and SRCH_WINDOW <= 120:
        return '91-120'
    elif SRCH_WINDOW >= 121 and SRCH_WINDOW <= 180:
        return '121-180'
    elif SRCH_WINDOW >= 181 and SRCH_WINDOW <= 240:
        return '181-240'
    else:
        return '>240'
    


In [83]:
#los bucket for booked los
def srchBookedLosBucket(booked_los):
    if booked_los is None:
        return '0'
    elif booked_los  in (0,1,2,3,4,5,6,7):
        return str(booked_los)
    elif booked_los >= 8 and booked_los <= 14:
        return '8-14'
    else:
        return '>14'

#bw bucket for booked bw
def srchBookedBwBucket(BOOKED_BKG_WINDOW):
    if BOOKED_BKG_WINDOW is None:
        return '0'
    elif BOOKED_BKG_WINDOW <= 0:
        return '<= 0'
    elif BOOKED_BKG_WINDOW == 1:
        return '1'
    elif BOOKED_BKG_WINDOW == 2:
        return '2'
    elif BOOKED_BKG_WINDOW >= 3 and BOOKED_BKG_WINDOW <= 4:
        return '3-4'
    elif BOOKED_BKG_WINDOW >= 5 and BOOKED_BKG_WINDOW <= 7:
        return '5-7'
    elif BOOKED_BKG_WINDOW >= 8 and BOOKED_BKG_WINDOW <= 14:
        return '8-14'
    elif BOOKED_BKG_WINDOW >= 15 and BOOKED_BKG_WINDOW <= 20:
        return '15-20'
    elif BOOKED_BKG_WINDOW >= 21 and BOOKED_BKG_WINDOW <= 30:
        return '21-30'
    elif BOOKED_BKG_WINDOW >= 31 and BOOKED_BKG_WINDOW <= 40:
        return '31-40'
    elif BOOKED_BKG_WINDOW >= 41 and BOOKED_BKG_WINDOW <= 60:
        return '41-60'
    elif BOOKED_BKG_WINDOW >= 61 and BOOKED_BKG_WINDOW <= 90:
        return '61-90'
    elif BOOKED_BKG_WINDOW >= 91 and BOOKED_BKG_WINDOW <= 120:
        return '91-120'
    elif BOOKED_BKG_WINDOW >= 121 and BOOKED_BKG_WINDOW <= 180:
        return '121-180'
    elif BOOKED_BKG_WINDOW >= 181 and BOOKED_BKG_WINDOW <= 240:
        return '181-240'
    else:
        return '>240'

In [84]:
#converting booking flag from string values to boolean
def booking_flag(booked_flag):
    if booked_flag == 'Y':
        return 1
    else:
        return 0

In [85]:
# Registering Functions as UDF's
udfsearchToCategory=udf(searchToCategory, StringType())
udfsearchbucketToCategory=udf(searchbucketToCategory, StringType())
udfsrchBookedLosBucket=udf(srchBookedLosBucket, StringType())
udfsrchsrchBookedBwBucket=udf(srchBookedBwBucket, StringType())
booking_flag_udf=udf(booking_flag, IntegerType())

# Importing Hotel Click Book Agg Table

In [86]:
from datetime import *
import time

date_list = [
"2016-02-14",
"2016-02-15",
"2016-02-16",
"2016-02-17",
"2016-02-18",
"2016-02-19",
"2016-02-20",
"2016-02-21",
"2016-02-22",
"2016-02-23",
"2016-02-24",
"2016-02-25",
"2016-02-26",
"2016-02-27",
"2016-02-28",
"2016-02-29",
"2016-03-01",
"2016-03-02",
"2016-03-03",
"2016-03-04",
"2016-03-05",
"2016-03-06",
"2016-03-07",
"2016-03-08",
"2016-03-09",
"2016-03-10",
"2016-03-11",
"2016-03-12",
"2016-03-13",
"2016-03-14",
"2016-03-15",
"2016-03-16",
"2016-03-17",
"2016-03-18",
"2016-03-19",
"2016-03-20",
"2016-03-21",
"2016-03-22",
"2016-03-23",
"2016-03-24",
"2016-03-25",
"2016-03-26",
"2016-03-27",
"2016-03-28",
"2016-03-29",
"2016-03-30",
"2016-03-31",
"2016-04-01",
"2016-04-02",
"2016-04-03",
"2016-04-04",
"2016-04-05",
"2016-04-06",
"2016-04-07",
"2016-04-08",
"2016-04-09",
"2016-04-10",
"2016-04-11",
"2016-04-12",
"2016-04-13",
"2016-04-14",
"2016-04-15",
"2016-04-16",
"2016-04-17",
"2016-04-18",
"2016-04-19",
"2016-04-20",
"2016-04-21",
"2016-04-22",
"2016-04-23",
"2016-04-24",
"2016-04-25",
"2016-04-26",
"2016-04-27",
"2016-04-28",
"2016-04-29",
"2016-04-30",
"2016-05-01",
"2016-05-02",
"2016-05-03",
"2016-05-04",
"2016-05-05",
"2016-05-06",
"2016-05-07",
"2016-05-08",
"2016-05-09",
"2016-05-10",
"2016-05-11",
"2016-05-12",
"2016-05-13",
"2016-05-14",
"2016-05-15",
"2016-05-16",
"2016-05-17",
"2016-05-18",
"2016-05-19",
"2016-05-20",
"2016-05-21",
"2016-05-22",
"2016-05-23",
"2016-05-24",
"2016-05-25",
"2016-05-26",
"2016-05-27",
"2016-05-28",
"2016-05-29",
"2016-05-30",
"2016-05-31",
"2016-06-01",
"2016-06-02",
"2016-06-03",
"2016-06-04",
"2016-06-05",
"2016-06-06",
"2016-06-07",
"2016-06-08",
"2016-06-09",
"2016-06-10",
"2016-06-11",
"2015-06-07",
"2015-06-08",
"2015-06-09",
"2015-06-10",
"2015-06-11",
"2015-06-12",
"2015-06-13"]


#schema for HCBA table
hcba_schema = StructType([
                            StructField('PARTNER_POS',StringType(), True),
                            StructField('DEVICE_TYPE',StringType(), True),
                            StructField('CLICKED_HOTEL_ID',IntegerType(), True),
                            StructField('SRCH_WINDOW',IntegerType(), True),
                            StructField('SRCH_LOS',IntegerType(), True),
                            StructField('TRAFFIC',IntegerType(), True),
                            StructField('COST',DoubleType(), True),
                            StructField('BOOKED_FLAG',StringType(), True),
                            StructField('BOOKED_HOTEL_ID',IntegerType(), True),
                            StructField('BOOKED_LOS',IntegerType(), True),
                            StructField('BOOKED_BKG_WINDOW',IntegerType(), True),
                            StructField('GROSS_PROFIT',DoubleType(), True),
                            StructField('PARTNER_ORG',StringType(), True),
                            StructField('ACTIVITY_DATE',StringType(), True)
                          ])

In [87]:
##CREATE EMPTY DATAFRAME 
hcba = sqlContext.createDataFrame(sc.emptyRDD(), hcba_schema)

In [88]:
##APPENDING THE DATA IN ABOVE CREATED hcba TABLE 
for dates in date_list:
    path_hcba = 's3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date='+dates+'/*'  
    
    try :
        temp_hcba = (sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferschema = True, delimiter = "\t",nullValue= '\\N')
                    .load(path_hcba)
                    .select('C1','C3','C7','C8','C9','C13','C14','C19','C20','C21','C22','C45','C65')
                    .filter("C65 = 'TRIPADVISOR' and C1 = 'CA' and C7 != '0' and C13 != '0'") 
                    .withColumn('ACTIVITY_DATE',lit(dates))
                    )
        hcba = hcba.unionAll(temp_hcba)
        
    except: 
        print (path_hcba)
        print('file does not exist')

# Cleaning and Outlier Treatment of HCBA Data

In [89]:
## Changing Data Type of HCBA Table
cols_int = ['SRCH_LOS','SRCH_WINDOW','TRAFFIC','BOOKED_BKG_WINDOW','BOOKED_LOS']
cols_float = ['COST','GROSS_PROFIT']

hcba_changed_data_type = data_type_change_float(data_type_change_int(hcba,cols_int),cols_float)

hcba_final = (hcba_changed_data_type
                .withColumn("SRCH_LOS_BUCKET", udfsearchToCategory("SRCH_LOS"))
                .withColumn("SRCH_BW_BUCKET", udfsearchbucketToCategory("SRCH_WINDOW"))
                .withColumn("BOOKED_LOS_BUCKET", udfsrchBookedLosBucket("BOOKED_LOS"))
                .withColumn("BOOKED_BW_BUCKET", udfsrchsrchBookedBwBucket("BOOKED_BKG_WINDOW"))
                .withColumn("BOOKING", booking_flag_udf("BOOKED_FLAG"))
                .repartition(1)
             )


In [90]:
#columns required in final cleaned hcba table
cols_required = ['PARTNER_POS','DEVICE_TYPE','HOTEL_ID','PARTNER_ORG','ACTIVITY_DATE','TRAFFIC','BOOKING_WINDOW','LOS',
                 'COST','GROSS_PROFIT','LOS_BUCKET','BW_BUCKET','BOOKING']

In [91]:
#replacing the wrongly populated search parameters for the search queries that are booked with booking parameters
hcba_replac_srch_with_booking = (hcba_final.filter("BOOKED_FLAG = 'Y'")
                                 .filter("SRCH_WINDOW < -1 or SRCH_LOS <= 0")
                                 .withColumnRenamed('BOOKED_BKG_WINDOW', 'BOOKING_WINDOW')
                                 .withColumnRenamed('BOOKED_BW_BUCKET', 'BW_BUCKET')
                                 .withColumnRenamed('BOOKED_LOS_BUCKET', 'LOS_BUCKET')
                                 .withColumnRenamed('BOOKED_HOTEL_ID', 'HOTEL_ID')
                                 .withColumnRenamed('BOOKED_LOS', 'LOS')
                                 .select(cols_required)
                            )

In [92]:
#filtering out the outliers from the seacrh data
hcba_srch_data_cleaned = (hcba_final.filter("BOOKED_FLAG= 'N' and SRCH_WINDOW >= -1 and SRCH_LOS > 0 and GROSS_PROFIT = 0.0")
                             .withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID')
                             .withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW')
                             .withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET')
                             .withColumnRenamed('SRCH_LOS', 'LOS')
                             .withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
                             .select(cols_required)
                            )


In [93]:
#filtering out the outilers for the booking data
hcba_booking_data_cleaned = (hcba_final.filter("BOOKED_FLAG= 'Y' and SRCH_WINDOW >= -1 and SRCH_LOS > 0")
                             .withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID')
                             .withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW')
                             .withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET')
                             .withColumnRenamed('SRCH_LOS', 'LOS')
                             .withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
                             .select(cols_required)
                            )

In [94]:
#Union of all the above 3 tables 
hcba_cleaned_data =( hcba_replac_srch_with_booking.unionAll(hcba_srch_data_cleaned)
                    .unionAll(hcba_booking_data_cleaned)
                    .repartition(1)
                    .cache())

In [95]:
hcba_cleaned_data.printSchema()

root
 |-- PARTNER_POS: string (nullable = true)
 |-- DEVICE_TYPE: string (nullable = true)
 |-- HOTEL_ID: string (nullable = true)
 |-- PARTNER_ORG: string (nullable = true)
 |-- ACTIVITY_DATE: string (nullable = true)
 |-- TRAFFIC: integer (nullable = true)
 |-- BOOKING_WINDOW: integer (nullable = true)
 |-- LOS: integer (nullable = true)
 |-- COST: double (nullable = true)
 |-- GROSS_PROFIT: double (nullable = true)
 |-- LOS_BUCKET: string (nullable = true)
 |-- BW_BUCKET: string (nullable = true)
 |-- BOOKING: integer (nullable = true)



# Subset the HCBA Table for Populating Average Weekly Traffic in Opti AD

In [96]:
# Only current years data
hcba_opti = (hcba_cleaned_data.filter("ACTIVITY_DATE >= '2016-01-01'").filter("ACTIVITY_DATE <= '2016-06-04'")# valid 1
                   .withColumn('WEEK',weekofyear("ACTIVITY_DATE"))) 

hcba_opti_agg = hcba_opti.groupBy('HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE')

hcba_opti_agg_total_traffic = (hcba_opti_agg.agg({'TRAFFIC':'sum','BOOKING':'sum','GROSS_PROFIT':'sum','COST':'sum'})
                                .withColumnRenamed('sum(TRAFFIC)','TRAFFIC')
                                .withColumnRenamed('sum(BOOKING)','BOOKING')
                                .withColumnRenamed('sum(GROSS_PROFIT)','GROSS_PROFIT')
                                .withColumnRenamed('sum(COST)','COST')
                                .repartition(1))

hcba_opti_agg_number_weeks =( hcba_opti_agg.agg(func.countDistinct('WEEK'))
                            .withColumnRenamed('count(WEEK)','NO_WEEKS')
                            .repartition(1))

hcba_opti_with_cols_traffic_weeks =(hcba_opti_agg_total_traffic
                                    .join(hcba_opti_agg_number_weeks,['HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE'],'inner')
                                    .filter("DEVICE_TYPE = 'DESKTOP'")
                                    .repartition(1))

hcba_opti_final1 = (hcba_opti_with_cols_traffic_weeks.withColumn('AVERAGE_WEEKLY_TRAFFIC',
                    hcba_opti_with_cols_traffic_weeks.TRAFFIC/hcba_opti_with_cols_traffic_weeks.NO_WEEKS)
                    .withColumn('CPC_HCBA',
                    hcba_opti_with_cols_traffic_weeks.COST/hcba_opti_with_cols_traffic_weeks.TRAFFIC)
                   .groupBy('HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE')
                   .agg({'AVERAGE_WEEKLY_TRAFFIC':'avg','CPC_HCBA':'avg'})
                   .withColumnRenamed("avg(AVERAGE_WEEKLY_TRAFFIC)","AVERAGE_WEEKLY_TRAFFIC")
                   .withColumnRenamed("avg(CPC_HCBA)","CPC_HCBA")
                   .filter("DEVICE_TYPE = 'DESKTOP'")
                   .drop("DEVICE_TYPE"))
hcba_opti_final = (hcba_opti_final1
                  .join(hcba_opti_with_cols_traffic_weeks,['HOTEL_ID','LOS_BUCKET','BW_BUCKET'],'inner')
                  .drop("NO_WEEKS")
                  .repartition(1)
                  .cache())

In [25]:
# # this is the initial code used in EXPEDIA_TOP_FINAL_SCRIPT, the above code is used to split into trianing and validaiton
# mbap_agg =(mbap.groupBy('HOTEL_ID','DEVICE_TYPE')
#             .agg({"SELLER_RANK":"avg","CPC":"avg"})
#             .withColumnRenamed("avg(SELLER_RANK)","SELLER_RANK")
#             .withColumnRenamed("avg(CPC)","CPC"))

# mbap_final = (mbap_agg.withColumn('SELLER_RANK_BUCKET',seller_rank_bucket_udf('SELLER_RANK'))
#               .select('HOTEL_ID','DEVICE_TYPE','SELLER_RANK','CPC','SELLER_RANK_BUCKET')
#               .repartition(1)
#               .cache())

# Importing THPA  

In [27]:
#UDF for GBV Buckets
def GBVbins(TOTAL_RATE_MIN):
    if TOTAL_RATE_MIN >= 0 and TOTAL_RATE_MIN < 115:
        return '0 - 115'
    elif TOTAL_RATE_MIN  >= 115 and TOTAL_RATE_MIN < 163:
        return '115 - 163'
    elif TOTAL_RATE_MIN  >= 163 and TOTAL_RATE_MIN < 244:
        return '163 - 244'
    elif TOTAL_RATE_MIN  >= 244 and TOTAL_RATE_MIN < 354:
        return '244 - 354'
    elif TOTAL_RATE_MIN  >= 354 and TOTAL_RATE_MIN < 606:
        return '354 - 606'
    else:
        return '606 - 5000'

In [28]:
udfGBVbinsCategory=udf(GBVbins, StringType())

In [29]:
from datetime import *
import time

In [30]:
date_list_thpa = ["2016-07-09","2016-07-10"]

In [31]:
TOP_schema = StructType([
                            StructField('DEVICE_TYPE',StringType(), True),
                            StructField('HOTEL_ID',IntegerType(), True),
                            StructField('BOOKING_WINDOW',IntegerType(), True),
                            StructField('LOS',IntegerType(), True),
                            StructField('ADULT_COUNT',DoubleType(), True), 
                            StructField('BASE_RATE_MIN',DoubleType(), True),
                            StructField('COMMISION_AVG',DoubleType(), True),
                            StructField('VARIABLE_MARGIN_AVG',DoubleType(), True),
                            StructField('LOCAL_DATE',StringType(), True),
                            StructField('TOTAL_RATE_MIN',StringType(), True)
        

                            ])

In [32]:
thpa = sqlContext.createDataFrame(sc.emptyRDD(), TOP_schema)

In [33]:
for dates in date_list_thpa:
    path_thpa = 's3n://ewe-core-meta-prod/CORE/TOP_HOTEL_PRICE_AGG/local_date='+dates+'/*'  
    print (path_thpa)
    try :
        temp_thpa = sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferschema = True, delimiter = "\t",nullValue= '\\N').load(path_thpa)
        temp_thpa = temp_thpa.filter((temp_thpa.C2 == 'DESKTOP')&(temp_thpa.C0 == 'CA')&(temp_thpa.C33 == 'EXPEDIA'))
        temp_thpa = temp_thpa.select('C2','C3','C9','C8','C10','C14','C28','C30','C32','C13')
        print('file exist')
        thpa = thpa.unionAll(temp_thpa)
    except: print('file does not exist')

s3n://ewe-core-meta-prod/CORE/TOP_HOTEL_PRICE_AGG/local_date=2016-07-09/*
file exist
s3n://ewe-core-meta-prod/CORE/TOP_HOTEL_PRICE_AGG/local_date=2016-07-10/*
file exist


In [34]:
thpa1 = thpa.distinct()
thpa_float = data_type_change_float(thpa1, ['BASE_RATE_MIN','COMMISION_AVG','VARIABLE_MARGIN_AVG','TOTAL_RATE_MIN'])
thpa_int = data_type_change_int(thpa_float, ['LOS','BOOKING_WINDOW'])

In [35]:
#thpa_opti3.printSchema()

In [36]:
thpa_opti_rm_nulls = thpa_int.dropna()

In [37]:
thpa_opti1 = thpa_opti_rm_nulls.withColumn("NET_REVENUE",(thpa_opti_rm_nulls.COMMISION_AVG+thpa_opti_rm_nulls.VARIABLE_MARGIN_AVG))
thpa_opti2 = thpa_opti1.groupBy("HOTEL_ID","LOS","BOOKING_WINDOW").agg({'NET_REVENUE':'avg',"BASE_RATE_MIN":"avg", "TOTAL_RATE_MIN":"avg"}).\
            withColumnRenamed('avg(NET_REVENUE)','NET_REVENUE').withColumnRenamed('avg(BASE_RATE_MIN)','BASE_RATE_MIN').withColumnRenamed('avg(TOTAL_RATE_MIN)','TOTAL_RATE_MIN')
thpa_opti3  = thpa_opti2.withColumn("LOS_BUCKET", udfsearchToCategory("LOS")).\
            withColumn("BW_BUCKET", udfsearchbucketToCategory("BOOKING_WINDOW"))

In [38]:
thpa_opti4 = thpa_opti3.groupBy("HOTEL_ID","LOS_BUCKET","BW_BUCKET").agg({"NET_REVENUE":"avg","BASE_RATE_MIN":"avg","TOTAL_RATE_MIN":"avg"}). \
            withColumnRenamed('avg(NET_REVENUE)','NET_REVENUE').withColumnRenamed('avg(BASE_RATE_MIN)','BASE_RATE_MIN')
thpa_opti5 = thpa_opti4.withColumn("NET_REVENUE_0",round(thpa_opti4.NET_REVENUE,2)).drop("NET_REVENUE")


In [39]:
thpa_opti5 = thpa_opti5.filter(thpa_opti5["avg(TOTAL_RATE_MIN)"] <= 5000.00)

In [40]:
thpa_opti5 = thpa_opti5.withColumnRenamed("avg(TOTAL_RATE_MIN)","TOTAL_RATE_MIN")

In [41]:
#thpa_opti5[~thpa_opti5.TOTAL_RATE_MIN.isNull()].count()

In [42]:
thpa_opti5 = thpa_opti5.withColumn("GBV_BUCKET",udfGBVbinsCategory('TOTAL_RATE_MIN'))

In [43]:
thpa_opti6 = thpa_opti5.filter(thpa_opti5["NET_REVENUE_0"] != 0.00).withColumnRenamed("NET_REVENUE_0","NET_REVENUE")

In [44]:
thpa_opti7 = thpa_opti6.filter(thpa_opti5["BASE_RATE_MIN"] != 0.00)

In [45]:
thpa_opti8 = thpa_opti7.withColumn('ratio',thpa_opti6.NET_REVENUE/thpa_opti6.BASE_RATE_MIN)

In [46]:
thpa_opti8 = thpa_opti8.filter(thpa_opti8["ratio"] <= 0.20)

In [47]:
thpa_opti8.printSchema()

root
 |-- HOTEL_ID: string (nullable = true)
 |-- LOS_BUCKET: string (nullable = true)
 |-- BW_BUCKET: string (nullable = true)
 |-- TOTAL_RATE_MIN: double (nullable = true)
 |-- BASE_RATE_MIN: double (nullable = true)
 |-- NET_REVENUE: double (nullable = true)
 |-- GBV_BUCKET: string (nullable = true)
 |-- ratio: double (nullable = true)



In [48]:
'''hcba_rf_union_train.join(hotel_dim_final,"HOTEL_ID","inner")
                       .join(mbap_final_train,['HOTEL_ID','DEVICE_TYPE'],'left_outer')'''

'hcba_rf_union_train.join(hotel_dim_final,"HOTEL_ID","inner")\n                       .join(mbap_final_train,[\'HOTEL_ID\',\'DEVICE_TYPE\'],\'left_outer\')'

In [49]:
final_join = hcba_cleaned_data.join(hotel_dim_final,"HOTEL_ID","inner").join(thpa_opti8,['HOTEL_ID','LOS_BUCKET','BW_BUCKET'],'left')

In [50]:
rf_training_data1 = final_join.filter("ACTIVITY_DATE >= '2016-06-28' and ACTIVITY_DATE <= '2016-07-18'")
rf_training_data2 = final_join.filter("ACTIVITY_DATE < '2016-01-01'")
rf_train_data_final = rf_training_data1.unionAll(rf_training_data2)
rf_valid_data_final = final_join.filter("ACTIVITY_DATE >= '2016-07-19'")


In [51]:
rf_train_data_final.printSchema()

root
 |-- HOTEL_ID: string (nullable = true)
 |-- LOS_BUCKET: string (nullable = true)
 |-- BW_BUCKET: string (nullable = true)
 |-- PARTNER_POS: string (nullable = true)
 |-- DEVICE_TYPE: string (nullable = true)
 |-- PARTNER_ORG: string (nullable = true)
 |-- ACTIVITY_DATE: string (nullable = true)
 |-- TRAFFIC: integer (nullable = true)
 |-- BOOKING_WINDOW: integer (nullable = true)
 |-- LOS: integer (nullable = true)
 |-- COST: double (nullable = true)
 |-- GROSS_PROFIT: double (nullable = true)
 |-- BOOKING: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- MODEL_NAME: string (nullable = true)
 |-- STAR_RATING_BUCKET: string (nullable = true)
 |-- HOTEL_TYPE_BUCKET: string (nullable = true)
 |-- MARKET_BUCKET: string (nullable = true)
 |-- REGION_BUCKET: string (nullable = true)
 |-- CITY_BUCKET: string (nullable = true)
 |-- TOTAL_RATE_MIN: double (nullable = true)
 |-- BASE_RATE_MIN: double (nullable = true)
 |-- NET_REVENUE: double (nullable = true)
 |-- GBV_

# Opti AD Creation


In [52]:
opti_search_query_universe = thpa_opti8

In [53]:
opti_search_query_universe = (opti_search_query_universe.select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","NET_REVENUE","GBV_BUCKET","TOTAL_RATE_MIN").repartition(1,"HOTEL_ID","LOS_BUCKET","BW_BUCKET").cache())

In [54]:
opti_search_query_universe_final = (opti_search_query_universe[opti_search_query_universe.NET_REVENUE.isNotNull()]
                                                  .join(hcba_opti_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],'left_outer')
                                                  .join(hotel_dim_final,"HOTEL_ID","inner")
                                                  .repartition(1)
                                                  .cache())

In [55]:
opti_search_query_universe_final.printSchema()

root
 |-- HOTEL_ID: string (nullable = true)
 |-- LOS_BUCKET: string (nullable = true)
 |-- BW_BUCKET: string (nullable = true)
 |-- NET_REVENUE: double (nullable = true)
 |-- GBV_BUCKET: string (nullable = true)
 |-- TOTAL_RATE_MIN: double (nullable = true)
 |-- CPC_HCBA: double (nullable = true)
 |-- AVERAGE_WEEKLY_TRAFFIC: double (nullable = true)
 |-- DEVICE_TYPE: string (nullable = true)
 |-- TRAFFIC: long (nullable = true)
 |-- COST: double (nullable = true)
 |-- GROSS_PROFIT: double (nullable = true)
 |-- BOOKING: long (nullable = true)
 |-- region: string (nullable = true)
 |-- MODEL_NAME: string (nullable = true)
 |-- STAR_RATING_BUCKET: string (nullable = true)
 |-- HOTEL_TYPE_BUCKET: string (nullable = true)
 |-- MARKET_BUCKET: string (nullable = true)
 |-- REGION_BUCKET: string (nullable = true)
 |-- CITY_BUCKET: string (nullable = true)



In [56]:
from pyspark.mllib.linalg import SparseVector, VectorUDT
import numpy
from pyspark.ml.feature import *
from pyspark.mllib.linalg import *
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer

In [57]:
name = 'CANADA'

if name == 'CANADA':
    region_level = 'CITY_BUCKET'
elif name == 'US':
    region_level = 'region'
elif name in ['Mexico', 'CARIBBEAN', 'North America']:
    region_level = 'MARKET_BUCKET'
else:
    region_level = 'REGION_BUCKET'

In [58]:
training_ad = (rf_train_data_final.filter(rf_train_data_final.MODEL_NAME == name)
               .filter("DEVICE_TYPE='DESKTOP'")
               .withColumnRenamed(region_level, 'REGION_LEVEL_BUCKET'))

optimization_test = (opti_search_query_universe_final.filter(opti_search_query_universe_final.MODEL_NAME == name)
                     .withColumnRenamed(region_level, 'REGION_LEVEL_BUCKET'))

In [59]:
optimization_test.printSchema()

root
 |-- HOTEL_ID: string (nullable = true)
 |-- LOS_BUCKET: string (nullable = true)
 |-- BW_BUCKET: string (nullable = true)
 |-- NET_REVENUE: double (nullable = true)
 |-- GBV_BUCKET: string (nullable = true)
 |-- TOTAL_RATE_MIN: double (nullable = true)
 |-- CPC_HCBA: double (nullable = true)
 |-- AVERAGE_WEEKLY_TRAFFIC: double (nullable = true)
 |-- DEVICE_TYPE: string (nullable = true)
 |-- TRAFFIC: long (nullable = true)
 |-- COST: double (nullable = true)
 |-- GROSS_PROFIT: double (nullable = true)
 |-- BOOKING: long (nullable = true)
 |-- region: string (nullable = true)
 |-- MODEL_NAME: string (nullable = true)
 |-- STAR_RATING_BUCKET: string (nullable = true)
 |-- HOTEL_TYPE_BUCKET: string (nullable = true)
 |-- MARKET_BUCKET: string (nullable = true)
 |-- REGION_BUCKET: string (nullable = true)
 |-- REGION_LEVEL_BUCKET: string (nullable = true)



In [60]:
# adding code for valdiation
validation_ad = (rf_valid_data_final.filter(rf_valid_data_final.MODEL_NAME == name)
               .filter("DEVICE_TYPE='DESKTOP'")
               .withColumnRenamed(region_level, 'REGION_LEVEL_BUCKET'))

In [61]:
# for training the data
# Aggregating the data to the required level
col_agg = ["BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET", "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET","GBV_BUCKET"]
training_ad_T = (training_ad.groupBy(col_agg)
                      .sum("TRAFFIC", "BOOKING","COST","GROSS_PROFIT")
                      .withColumnRenamed('sum(TRAFFIC)', 'TRAFFIC')
                      .withColumnRenamed('sum(BOOKING)', 'BOOKING')
                      .withColumnRenamed('sum(COST)', 'COST')
                      .withColumnRenamed('sum(GROSS_PROFIT)', 'GROSS_PROFIT'))
validation_ad_T = (validation_ad.groupBy(col_agg)
                      .sum("TRAFFIC", "BOOKING","COST","GROSS_PROFIT")
                      .withColumnRenamed('sum(TRAFFIC)', 'TRAFFIC')
                      .withColumnRenamed('sum(BOOKING)', 'BOOKING')
                      .withColumnRenamed('sum(COST)', 'COST')
                      .withColumnRenamed('sum(GROSS_PROFIT)', 'GROSS_PROFIT'))

In [62]:
# Filtering out the TRAFFIC (Outliers) from data
training_ad_T1 = (training_ad_T[~((training_ad_T.TRAFFIC <= 4) & (training_ad_T.BOOKING > 0))]
                               [~((training_ad_T.TRAFFIC == 5) & (training_ad_T.BOOKING == 2))]
                               [~((training_ad_T.TRAFFIC == 6) & (training_ad_T.BOOKING == 2))])

In [63]:
col_required_rf = ["MODEL_NAME","HOTEL_ID","LOS_BUCKET","BW_BUCKET","DEVICE_TYPE","STAR_RATING_BUCKET","GBV_BUCKET",
                   "HOTEL_TYPE_BUCKET","REGION_LEVEL_BUCKET","AVERAGE_WEEKLY_TRAFFIC",
                   "BASE_RATE_MIN","NET_REVENUE","TOTAL_RATE_MIN","TRAFFIC","BOOKING","chk_data","CPC_HCBA","GROSS_PROFIT","COST",'ACTIVITY_DATE']

In [64]:
training_ad_T2 = (training_ad_T1.withColumn("chk_data",lit("train"))
                                .withColumn('MODEL_NAME',lit(name))
                                .withColumn('HOTEL_ID',lit("NA"))
                                .withColumn('DEVICE_TYPE',lit("DESKTOP"))
                                .withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0.0))
                                .withColumn('TOTAL_RATE_MIN',lit(0.0))
                                .withColumn('CPC',lit(0.0))
                                .withColumn('BASE_RATE_MIN',lit(0.00))
                                .withColumn('NET_REVENUE',lit(0.00))
                                .withColumn('REGION_BUCKET',lit("NA"))
                                .withColumn('CPC_HCBA',lit(0.0))
                                .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                .select(col_required_rf))
                                

In [65]:
optimization_test2 = (optimization_test.withColumn('BASE_RATE_MIN',lit(0.00))
                                      .withColumn("DEVICE_TYPE",lit("DESKTOP"))
                                      .withColumn("chk_data",lit("test"))
                                      .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                      .select(col_required_rf))

In [66]:
# col_for_select = ["HOTEL_ID","BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET", "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET","SELLER_RANK_BUCKET"\
#                  ,"TRAFFIC","BOOKING","COST"]
# training_for_test = training_ad.select(col_for_select)
# validation_for_test = validation_ad.select(col_for_select)

training_for_test = training_ad_T # .select(col_for_select)
validation_for_test = validation_ad_T # .select(col_for_select)

In [67]:
col_required_results = ["MODEL_NAME","HOTEL_ID","LOS_BUCKET","BW_BUCKET","DEVICE_TYPE","STAR_RATING_BUCKET","GBV_BUCKET",
                   "HOTEL_TYPE_BUCKET","REGION_LEVEL_BUCKET","AVERAGE_WEEKLY_TRAFFIC",
                   "BASE_RATE_MIN","NET_REVENUE","TOTAL_RATE_MIN","TRAFFIC","BOOKING","chk_data","CPC_HCBA","GROSS_PROFIT","COST",'ACTIVITY_DATE']

In [68]:
# for testing on training data
training_for_test1 = (training_for_test.withColumn("chk_data",lit("train-testing"))
                                .withColumn('MODEL_NAME',lit(name))
                                .withColumn('HOTEL_ID',lit("NA"))
                                .withColumn('DEVICE_TYPE',lit("DESKTOP"))
                                .withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0.0))
                                .withColumn('TOTAL_RATE_MIN',lit(0.0))
                                .withColumn('CPC',lit(0.0))
                                .withColumn('BASE_RATE_MIN',lit(0.00))
                                .withColumn('NET_REVENUE',lit(0.00))
                                .withColumn('REGION_BUCKET',lit("NA"))
                                .withColumn('CPC_HCBA',lit(0.0))
                                .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                .select(col_required_results))

In [69]:
# for testing on validation data
validation_for_test1 = (validation_for_test.withColumn("chk_data",lit("validation-testing"))
                                .withColumn('MODEL_NAME',lit(name))
                                .withColumn('HOTEL_ID',lit("NA"))
                                .withColumn('DEVICE_TYPE',lit("DESKTOP"))
                                .withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0.0))
                                .withColumn('TOTAL_RATE_MIN',lit(0.0))
                                .withColumn('CPC',lit(0.0))
                                .withColumn('BASE_RATE_MIN',lit(0.00))
                                .withColumn('NET_REVENUE',lit(0.00))
                                .withColumn('REGION_BUCKET',lit("NA"))
                                .withColumn('CPC_HCBA',lit(0.0))
                                .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                .select(col_required_results))

In [70]:
test_train_data = (optimization_test2.unionAll(training_ad_T2)
                                    .unionAll(training_for_test1)
                                    .unionAll(validation_for_test1)
                                    .repartition(16)
                                    .cache())

In [71]:
# STRING INDEXING: Converting the data type of required columns to factor
def indexStringColumns(df, cols):
    tempdf = df
    for col in cols:
        stringIndexer = StringIndexer(inputCol=col, outputCol=col + "-num")
        si_model = stringIndexer.fit(tempdf)
        tempdf = si_model.transform(tempdf)
    return tempdf

In [ ]:
cols_rf_operations = {"BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET","STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET","GBV_BUCKET"}
dfnumeric = indexStringColumns(test_train_data, cols_rf_operations)

In [2]:
# One HOT Encoding
def oneHotEncodeColumns(df, cols):
    tempdf = df
    for col in cols:
        onehotenc = OneHotEncoder(
            inputCol=col + "-num", outputCol=col + "-onehot")
        tempdf = onehotenc.transform(tempdf).drop(col + "-num")
    return tempdf

In [ ]:
test_train_data_class = oneHotEncodeColumns(dfnumeric, cols_rf_operations)

In [ ]:
cols_rf_model = ["BW_BUCKET","LOS_BUCKET","HOTEL_TYPE_BUCKET","STAR_RATING_BUCKET","REGION_LEVEL_BUCKET","GBV_BUCKET" 
                "TRAFFIC","BOOKING","STAR_RATING_BUCKET-onehot","REGION_LEVEL_BUCKET-onehot","BW_BUCKET-onehot", 
                "HOTEL_TYPE_BUCKET-onehot","LOS_BUCKET-onehot","GBV_BUCKET-onehot"]

In [ ]:
training_ad_T1_Class = (test_train_data_class.filter(test_train_data_class.chk_data=="train")
                        .drop("chk_data")
                        .select(cols_rf_model))


In [ ]:
# Transforming the training dataset
# Defining function for the same
def genRows(row):
    TRAFFIC = row.TRAFFIC
    BOOKING = row.BOOKING
    BOOKINGcounter = 0
    rowList = []

    for i in range(TRAFFIC):
        row1 = row[:]
        row1 = list(row1)
        row1.TRAFFIC = 1
        row1 = tuple(row1)
        if BOOKINGcounter < int(BOOKING):
            row1 = list(row1)
            row1.BOOKING = 1
            row1 = tuple(row1)
        else:
            row1 = list(row1)
            row1.BOOKING = 0
            row1 = tuple(row1)
        BOOKINGcounter += 1
        rowList.append(row1)
    return rowList

In [ ]:
training_ad_T1_Class_rdd = training_ad_T1_Class.rdd
training_ad_T1_Class_rdd = training_ad_T1_Class_rdd.flatMap(lambda x: genRows(x))

In [ ]:
training_ad_T1_Class.printSchema()

In [ ]:
# Reconverting to dataframe
from pyspark.mllib.linalg import SparseVector, VectorUDT

schema = StructType([StructField('BW_BUCKET', StringType(), True),
                     StructField('LOS_BUCKET', StringType(), True),
                     StructField('HOTEL_TYPE_BUCKET', StringType(), True),
                     StructField('STAR_RATING_BUCKET', StringType(), True),
                     StructField('REGION_LEVEL_BUCKET',StringType(), True),
                     StructField('GBV_BUCKET',StringType(), True),
                     StructField('TRAFFIC', IntegerType(), True),
                     StructField('BOOKING', IntegerType(), True),
                     StructField('STAR_RATING_BUCKET-onehot',VectorUDT(), True),
                     StructField('REGION_LEVEL_BUCKET-onehot',VectorUDT(), True),
                     StructField('BW_BUCKET-onehot',  VectorUDT(), True),
                     StructField('HOTEL_TYPE_BUCKET-onehot',  VectorUDT(), True),
                     StructField('LOS_BUCKET-onehot',  VectorUDT(), True),
                     StructField('GBV_BUCKET-onehot', VectorUDT(), True),])

training_ad_T1_Class_df = sqlContext.createDataFrame(training_ad_T1_Class_rdd, schema)

In [ ]:
# Converting label to double datatype
toDouble = udf(lambda x: float(x * 1.00), DoubleType())

training_ad_T1_Class_df = training_ad_T1_Class_df.withColumn('BOOKING', toDouble(training_ad_T1_Class_df.BOOKING))

In [ ]:
# Taking only relevant columns as features
colList = training_ad_T1_Class_df.columns
colList.remove('BOOKING')
colList.remove('TRAFFIC')
colList.remove('BW_BUCKET')
colList.remove('LOS_BUCKET')
colList.remove('HOTEL_TYPE_BUCKET')
colList.remove('STAR_RATING_BUCKET')
colList.remove('REGION_LEVEL_BUCKET')
colList.remove('GBV_BUCKET')

vecAssembler = VectorAssembler(inputCols=colList, outputCol="features")

cols_train_ad_rf = ["BOOKING", "TRAFFIC", "BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET",
                    "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET","GBV_BUCKET", "features"]

train_ad = (vecAssembler.transform(training_ad_T1_Class_df)
                        .select(cols_train_ad_rf)
                        .withColumnRenamed("BOOKING", "label")
                        .repartition(1)
                        .cache())

In [ ]:
col_remove = ["STAR_RATING_BUCKET-onehot","REGION_LEVEL_BUCKET-onehot","BW_BUCKET-onehot",
              "HOTEL_TYPE_BUCKET-onehot","LOS_BUCKET-onehot","GBV_BUCKET-onehot"]

optimization_onehot = (test_train_data_class.filter(test_train_data_class.chk_data=="test")
                                            .drop("chk_data"))

vecAssembler = VectorAssembler(inputCols=colList, outputCol="indexedFeatures")

optimization_df_onehot = vecAssembler.transform(optimization_onehot)
                          
optimization_df = (optimization_df_onehot.select([col for col in optimization_df_onehot.columns if col not in col_remove])
                  .repartition(50)
                  .cache())

In [ ]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(train_ad)

featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(train_ad)

rf = RandomForestRegressor(labelCol="indexedLabel", featuresCol="indexedFeatures", featureSubsetStrategy="all",
                           numTrees=1000, impurity="variance", maxBins=32,maxDepth=4)

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

model = pipeline.fit(train_ad)

RF_Model = model.stages[2]

In [ ]:
optimization_df_predict = (RF_Model.transform(optimization_df)
                           .drop("indexedFeatures")
                           .withColumnRenamed("prediction", "CONVERSION_RATE")
                           .repartition(16))

In [ ]:
'''col_remove = ["STAR_RATING_BUCKET-onehot","REGION_LEVEL_BUCKET-onehot","BW_BUCKET-onehot","HOTEL_TYPE_BUCKET-onehot",
                "LOS_BUCKET-onehot","GBV_BUCKET-onehot"]

optimization_t_final1 = optimization_df_predict.select([col for col in optimization_df_predict.columns if col not in col_remove ])
'''
region_df_for_opti = (optimization_df_predict.filter(optimization_df_predict['MODEL_NAME']==name)
                      .cache())

# Optimization Algorithm

In [182]:
##Pulp package for Linear Programming Optimization algorithm
import pulp
from pulp import *

In [183]:
#if conversion rate is less then 0.0005 then taking it as 0
def predicted(predicted):
    if predicted < 0.0005:
        predicted = 0.0
    else:
        predicted
    return predicted

##If Net revenue is negative then making it positive 
def net_revenue(net_revenue):
    if net_revenue < 0.0 :
        net_revenue = net_revenue*(-1)
    else:
        net_revenue
    return net_revenue

##creating unique id for each search query
def var_name(id):
    return "T_" + str(id)

In [338]:
##Creating the objective function at search query level, combining it while running lp_solve algorithm
def objective_fun(net_revenue,predicted):
    return net_revenue * predicted

##Lower limit for the average weekly Traffic
def lower_limit_fun(net_revenue,average_weekly_traffic):
    if net_revenue >= 100.00:
        limit = 0.5*average_weekly_traffic
    elif net_revenue >= 50.00 and net_revenue < 100.00:
        limit = 0.2*average_weekly_traffic
    elif net_revenue >= 20.00 and net_revenue < 50.00:
        limit = 0.8*average_weekly_traffic    
    else:
        limit=0.8*average_weekly_traffic
        
    return limit

In [339]:
##registering the above defined functions as uoptimization_rf
objective_fun_udf = udf(objective_fun, DoubleType())
lower_limit_fun_udf = udf(lower_limit_fun, DoubleType())
var_name_udf = udf(var_name,StringType())
net_revenue_udf = udf(net_revenue, DoubleType())
predicted_udf = udf(predicted , DoubleType())

In [340]:
#Filtering rows with Traffic > 0 
optimization_rf_algo = region_df_for_opti[~region_df_for_opti.AVERAGE_WEEKLY_TRAFFIC.isNull()]

#Applying Prediction UDF for rounding CR less than 0.0005
region_df_algo_treated = (optimization_rf_algo.withColumn("predicted_new",predicted_udf("CONVERSION_RATE"))
                            .withColumn("net_revenue_new",net_revenue_udf("NET_REVENUE")))

region_df_algo_final= (region_df_algo_treated.withColumn("objective_fun",objective_fun_udf("net_revenue_new" ,"predicted_new"))
                            .withColumn("Lower_Limit", lower_limit_fun_udf("net_revenue_new" , "AVERAGE_WEEKLY_TRAFFIC"))
                            .withColumn("id", monotonicallyIncreasingId())
                            .withColumn("var_name" ,var_name_udf("id"))
                            .rdd
                            .repartition(1)
                            .cache())

region_count = region_df_algo_final.count()

In [341]:
##list of variuable names for opti algo
t_list = region_df_algo_final.map(lambda x  : x.var_name).collect()

##objective function for lp solve
objective_list =  region_df_algo_final.map(lambda x  : x.objective_fun).collect()
final_objective = {t_list[i]: objective_list[i] for i in range(0, region_count)}

##lower and upper limit 
lower_limit_list =  region_df_algo_final.map(lambda x  : x.Lower_Limit).collect()
upper_limit_list =  region_df_algo_final.map(lambda x  : x.AVERAGE_WEEKLY_TRAFFIC).collect()
lower_limit_final = {t_list[i]: lower_limit_list[i] for i in range(0, region_count)}
upper_limit_final = {t_list[i]: upper_limit_list[i] for i in range(0, region_count)}

In [342]:
## Optimization Algorithm
cnt = 0.5
lp_status = "Start"
region_name = region_df_algo_final.first()[0]

while ((lp_status != "Optimal")  & (cnt <=5.0)):
    a = cnt
    
    def constraint_fun(row):
        last_row = row.CPC_HCBA - a * row.objective_fun
        return last_row
    
    last_row = region_df_algo_final.map(lambda x: constraint_fun(x)).collect()
    
    final_constraint = {t_list[i]: last_row[i] for i in range(0, region_count)}
    
    prob = LpProblem("The Optimization Problem", LpMaximize)
    
    lp_var = {}
    for variable in t_list:
        variable = LpVariable(variable,lower_limit_final[variable],upper_limit_final[variable])
        lp_var[str(variable)] = variable
    
    prob += lpSum([final_objective[i]*lp_var[i] for i in t_list])
    
    prob += lpSum([final_constraint[i]*lp_var[i] for i in t_list]) == 0
    
    prob.solve()
    
    lp_status = LpStatus[prob.status]
    
    var_values = {"start":0}
    for v in prob.variables():
        var_values.update({v.name[2:]:v.varValue})
   
    results={"region_name":region_name,"lp_status":LpStatus[prob.status],"cnt":(cnt), "gp":value(prob.objective),"var_list":var_values}
    
    cnt += 0.1
    
    print(cnt)

0.6
0.7
0.7999999999999999
0.8999999999999999
0.9999999999999999
1.0999999999999999
1.2
1.3
1.4000000000000001
1.5000000000000002
1.6000000000000003
1.7000000000000004
1.8000000000000005
1.9000000000000006
2.0000000000000004
2.1000000000000005
2.2000000000000006
2.3000000000000007
2.400000000000001
2.500000000000001
2.600000000000001
2.700000000000001
2.800000000000001
2.9000000000000012
3.0000000000000013
3.1000000000000014
3.2000000000000015
3.3000000000000016


In [343]:
results

{'cnt': 3.2000000000000015,
 'gp': 34611.873558924795,
 'lp_status': 'Optimal',
 'region_name': 'US',
 'var_list': {'94489283870': 0.8,
  '68719480001': 0.8,
  '4334': 0.8,
  '34359739171': 2.0,
  '77309415399': 0.2,
  '111669153927': 1.0,
  '120259088375': 0.8,
  '3665': 3.0,
  '103079215450': 1.0,
  '85899346728': 1.0,
  '34359742408': 1.6666667,
  '17179872054': 1.0,
  '68719476778': 4.0,
  '120259085698': 1.0,
  '25769806913': 0.8,
  '77309413797': 2.0,
  '25769805014': 2.0,
  '120259085314': 0.8,
  '77309412585': 0.8,
  '68719480953': 1.0,
  '85899349313': 0.8,
  '17179872337': 2.0,
  '128849019519': 1.0,
  '25769808091': 1.0,
  '17179872678': 1.0,
  '128849022442': 1.0,
  '25769806847': 4.2,
  '60129543890': 1.0,
  '103079217922': 1.0,
  '85899347784': 0.8,
  '103079217263': 1.0,
  '94489284414': 1.6,
  '111669150373': 2.5,
  '68719479573': 0.2,
  '60129543792': 2.0,
  '85899347078': 4.3333333,
  '17179871098': 1.6666667,
  '8589935944': 1.6,
  '128849021057': 0.2,
  '68719479836

In [344]:
final_results= [results]

In [345]:
def df_create(x):
    mydictionary = x['var_list']
    Efficiency =  x['cnt']
    Model_Name = x['region_name'][0]
    
    var_list = []
    for key in mydictionary:
        var_list.append((key, mydictionary[key],Efficiency,Model_Name))
          
    df = pandas.DataFrame(var_list, columns=['var_id', 'var_value','Efficiency','Model'])
    df_final = sqlContext.createDataFrame(df)
    
    return df_final 



In [346]:
import pandas
region_df_algo_final = sqlContext.createDataFrame(region_df_algo_final) #changing back to sqlcontext

## Generating Results from Throttles

In [347]:
field = [StructField("var_id", StringType(), True),StructField("var_value", DoubleType(), True),StructField("Efficiency", FloatType(), True),StructField("Model", StringType(), True)]
schema = StructType(field)

optimization_df_algo1 = sqlContext.createDataFrame(sc.emptyRDD(), schema)

for i in range(len(final_results)):
    optimization_df_algo1 = optimization_df_algo1.unionAll(df_create(final_results[i]))

optimization_df_algo1 = optimization_df_algo1.cache()
#final_results.unpersist()

optimization_df_algo1 = optimization_df_algo1.withColumn("id",optimization_df_algo1["var_id"].cast(LongType()))
optimization_df_algo2 = region_df_algo_final.select([col for col in region_df_algo_final.columns if col not in {"predicted_new","net_revenue_new","objective_fun","Lower_Limit","var_name","part_id"}]).join(optimization_df_algo1,"id","inner")

In [348]:
# here is where nathan is picking up on the code, going to include both training and validation sets
def threshold(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (1-z)

threshold_udf = udf(threshold,DoubleType())

def traffic_allow(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (z)

traffic_allow_udf = udf(traffic_allow,DoubleType())

In [349]:
optimization_df_algo3 = optimization_df_algo2.withColumn("threshold",threshold_udf("var_value","AVERAGE_WEEKLY_TRAFFIC")).                        withColumn("traffic_allow",traffic_allow_udf("var_value","AVERAGE_WEEKLY_TRAFFIC"))

optimization_df_algo3 = optimization_df_algo3.select([col for col in optimization_df_algo3.columns if col not in {"id","var_id","Model"}])
optimization_df_algo3 = optimization_df_algo3.withColumnRenamed("var_value","Lp_solution")
optimization_df_algo_final = optimization_df_algo3.select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","threshold","NET_REVENUE","traffic_allow")


In [350]:
hcba_updatedcols = (hcba_opti_final.select('HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE','CPC_HCBA','AVERAGE_WEEKLY_TRAFFIC')
                                .withColumnRenamed('AVERAGE_WEEKLY_TRAFFIC','AVERAGE_WEEKLY_TRAFFIC-usedintraining')
                                .withColumnRenamed('CPC_HCBA','CPC_HCBA-usedintraining'))

In [351]:
# the data is already in a spark dataframe, no need to read in as pandas and then convert
# using validation data first
# validation_df_spark = rf_validation_data
validation_df_spark = validation_ad.join(hcba_updatedcols,['HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE'],'left_outer') #changing so that we have the predicted conversion rate as well
validation_df_spark1=validation_df_spark.filter(validation_df_spark.DEVICE_TYPE=='DESKTOP')

In [352]:
def change_los(los):
    if los =="14-Aug":
        return "8-14"
    else :
        return los
change_los_udf = udf(change_los,StringType())

def change_bw(bw):
    if bw =="14-Aug":
        return "8-14"
    elif bw == "<=0":
        return "0"
    elif bw == "4-Mar":
        return "3-4"
    elif bw == "7-May":
        return "5-7"
    else :
        return bw
change_bw_udf = udf(change_bw,StringType())


In [353]:
validation_df_spark2 = validation_df_spark1.withColumn("LOS_BUCKET_1",change_los_udf("LOS_BUCKET")).withColumn("BW_BUCKET_1",change_bw_udf("BW_BUCKET")).drop("LOS_BUCKET").drop("BW_BUCKET").withColumnRenamed("LOS_BUCKET_1","LOS_BUCKET").withColumnRenamed("BW_BUCKET_1","BW_BUCKET")

validation_df_spark3 = validation_df_spark2.filter(validation_df_spark2.MODEL_NAME == name)


In [354]:
common_queries = validation_df_spark3.join(optimization_df_algo_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")

valid_df = common_queries.withColumn("throttle_traffic",(common_queries.traffic_allow*common_queries.TRAFFIC))

valid_df = valid_df.withColumn("throttle_cost",((valid_df.COST*valid_df.throttle_traffic)/valid_df.TRAFFIC))
valid_df = valid_df.withColumn("average_booking",(valid_df.BOOKING*valid_df.traffic_allow))
valid_df = valid_df.withColumn("average_earning",((valid_df.GROSS_PROFIT*valid_df.average_booking)/valid_df.BOOKING))

valid_df.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"}).show()

valid_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"}).show()


+------------+------------------+------------------+------------+
|sum(TRAFFIC)|         sum(COST)| sum(GROSS_PROFIT)|sum(BOOKING)|
+------------+------------------+------------------+------------+
|        6718|2969.0384360326375|5931.2622999999985|          75|
+------------+------------------+------------------+------------+

+---------------------+------------------+--------------------+--------------------+
|sum(throttle_traffic)|sum(throttle_cost)|sum(average_earning)|sum(average_booking)|
+---------------------+------------------+--------------------+--------------------+
|    6028.199998957578|  2628.78913435251|  5279.8197606149215|   66.80000000401462|
+---------------------+------------------+--------------------+--------------------+



In [355]:
6572.329448118207/8203.2709

0.8011840067500645

In [100]:
4288.747653943791/5890.364661298798

0.7280954406985972

In [ ]:
# using training data second
training_df_spark = training_ad.join(hcba_updatedcols,['HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE'],'left_outer')
training_df_spark1= training_df_spark.filter(training_df_spark.DEVICE_TYPE=='DESKTOP').filter(training_df_spark.ACTIVITY_DATE>'2016-01-01')

In [ ]:
training_df_spark2 = training_df_spark1.withColumn("LOS_BUCKET_1",change_los_udf("LOS_BUCKET")).withColumn("BW_BUCKET_1",change_bw_udf("BW_BUCKET")).drop("LOS_BUCKET").drop("BW_BUCKET").withColumnRenamed("LOS_BUCKET_1","LOS_BUCKET").withColumnRenamed("BW_BUCKET_1","BW_BUCKET")

training_df_spark3 = training_df_spark2.filter(training_df_spark2.MODEL_NAME == name)

In [ ]:
common_queries_train = training_df_spark3.join(optimization_df_algo_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")

train_df = common_queries_train.withColumn("throttle_traffic",(common_queries_train.traffic_allow*common_queries_train.TRAFFIC))

train_df = train_df.withColumn("throttle_cost",((train_df.COST*train_df.throttle_traffic)/train_df.TRAFFIC))
train_df = train_df.withColumn("average_booking",(train_df.BOOKING*train_df.traffic_allow))
train_df = train_df.withColumn("average_earning",((train_df.GROSS_PROFIT*train_df.average_booking)/train_df.BOOKING))

train_df.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"}).show()

train_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"}).show()

In [ ]:
#till here

## for rule-based throttling


In [289]:
optimization_rf_rule_base = region_df_for_opti[region_df_for_opti.AVERAGE_WEEKLY_TRAFFIC.isNull()]
optimization_traffic_score =  (optimization_rf_rule_base.withColumn("SCORE",(optimization_rf_rule_base.EPB*optimization_rf_rule_base.CONVERSION_RATE))
                              .repartition(18)
                              .cache())

In [290]:
def throttle(score):
    if score <= 0.15:
        return 0.5
    elif (score < 0.65) & (score > 0.15):
        return 0.9
#    elif score >= 0.65:
#        return 0.75
    else:
        return 0.5


throttle_udf = udf(throttle,DoubleType())

In [291]:
optimization_traffic_throttle  = optimization_traffic_score.withColumn("threshold",throttle_udf(optimization_traffic_score.EPB))

optimization_df_rule_base_final = (optimization_traffic_throttle.withColumn("traffic_allow",(1-optimization_traffic_throttle.threshold))
                                  .select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","threshold","EPB","traffic_allow"))


In [292]:
common_queries_rule = validation_df_spark3.join(optimization_df_rule_base_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")

common_queries_rule_train = training_df_spark3.join(optimization_df_rule_base_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")

NameError: name 'training_df_spark3' is not defined

In [293]:
valid_df_rule = (common_queries_rule
                            .withColumn("throttle_traffic",(common_queries_rule.traffic_allow*common_queries_rule.TRAFFIC))
                            .cache())

In [294]:
valid_df_rule = valid_df_rule.withColumn("throttle_cost",((valid_df_rule.COST*valid_df_rule.throttle_traffic)/valid_df_rule.TRAFFIC))
valid_df_rule = valid_df_rule.withColumn("average_booking",(valid_df_rule.BOOKING*valid_df_rule.traffic_allow))
valid_df_rule = valid_df_rule.withColumn("average_earning",((valid_df_rule.GROSS_PROFIT*valid_df_rule.average_booking)/valid_df_rule.BOOKING))

valid_df_rule.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"}).show()

valid_df_rule.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"}).show()

+------------+------------------+-----------------+------------+
|sum(TRAFFIC)|         sum(COST)|sum(GROSS_PROFIT)|sum(BOOKING)|
+------------+------------------+-----------------+------------+
|       10164|10129.817960279921|       13325.4338|         220|
+------------+------------------+-----------------+------------+

+---------------------+------------------+--------------------+--------------------+
|sum(throttle_traffic)|sum(throttle_cost)|sum(average_earning)|sum(average_booking)|
+---------------------+------------------+--------------------+--------------------+
|               5082.0|5064.9089801399605|           6662.7169|               110.0|
+---------------------+------------------+--------------------+--------------------+



In [190]:
# training data returns null for rule-based because all of these cases have seen at least 1 avg weekly traffic
#train_df_rule = common_queries_rule_train.withColumn("throttle_traffic",(common_queries_rule_train.traffic_allow*common_queries_rule_train.TRAFFIC))

#train_df_rule = train_df_rule.withColumn("throttle_cost",((train_df_rule.COST*train_df_rule.throttle_traffic)/train_df_rule.TRAFFIC))
#train_df_rule = train_df_rule.withColumn("average_booking",(train_df_rule.BOOKING*train_df_rule.traffic_allow))
#train_df_rule = train_df_rule.withColumn("average_earning",((train_df_rule.GROSS_PROFIT*train_df_rule.average_booking)/train_df_rule.BOOKING))

#train_df_rule.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"}).show()

#train_df_rule.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"}).show()

In [ ]:
'''import pulp
from pulp import *

def change_los(los):
    if los =="14-Aug":
        return "8-14"
    else :
        return los
change_los_udf = udf(change_los,StringType())

def change_bw(bw):
    if bw =="14-Aug":
        return "8-14"
    elif bw == "<=0":
        return "0"
    elif bw == "4-Mar":
        return "3-4"
    elif bw == "7-May":
        return "5-7"
    else :
        return bw
change_bw_udf = udf(change_bw,StringType())

def df_create(x):
    mydictionary = x['var_list']
    Efficiency =  x['cnt']
    Model_Name = x['region_name'][0]
    
    var_list = []
    for key in mydictionary:
        var_list.append((key, mydictionary[key],Efficiency,Model_Name))
          
    df = pandas.DataFrame(var_list, columns=['var_id', 'var_value','Efficiency','Model'])
    df_final = sqlContext.createDataFrame(df)
    
    return df_final 
	
# here is where nathan is picking up on the code, going to include both training and validation sets
def threshold(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (1-z)

threshold_udf = udf(threshold,DoubleType())

def traffic_allow(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (z)

traffic_allow_udf = udf(traffic_allow,DoubleType())

#if conversion rate is less then 0.0005 then taking it as 0
def predicted(predicted):
    if predicted < 0.0005:
        predicted = 0.0
    else:
        predicted
    return predicted

##If Net revenue is negative then making it positive 
def net_revenue(net_revenue):
    if net_revenue < 0.0 :
        net_revenue = net_revenue*(-1)
    else:
        net_revenue
    return net_revenue

##creating unique id for each search query
def var_name(id):
    return "T_" + str(id)
	
##Creating the objective function at search query level, combining it while running lp_solve algorithm
def objective_fun(net_revenue,predicted):
    return net_revenue * predicted
	

##registering the above defined functions as uoptimization_rf
objective_fun_udf = udf(objective_fun, DoubleType())
var_name_udf = udf(var_name,StringType())
net_revenue_udf = udf(net_revenue, DoubleType())
predicted_udf = udf(predicted , DoubleType())

#Filtering rows with Traffic > 0 
optimization_rf_algo = region_df_for_opti[~region_df_for_opti.AVERAGE_WEEKLY_TRAFFIC.isNull()]

#Applying Prediction UDF for rounding CR less than 0.0005
region_df_algo_treated = (optimization_rf_algo.withColumn("predicted_new",predicted_udf("CONVERSION_RATE"))
                            .withColumn("net_revenue_new",net_revenue_udf("EPB")))

####change the limit ranges if required
cons_epb100 = [0.2,0.5,0.8]
cons_epb50_100 = [0.2,0.5,0.8]
cons_epb20_50 =  [0.2,0.5,0.8]
cons_epb20 = [0.2,0.5,0.8]

counter = 1

for l1 in cons_epb100:
	for l2 in cons_epb50_100:
		for l3 in cons_epb20_50:
			for l4 in cons_epb20:
							
				if (counter >=1) & (counter <=30): #change it to the first number and last number from which starting to run
				##Lower limit for the average weekly Traffic
					def lower_limit_fun(net_revenue,average_weekly_traffic):
						if net_revenue >= 100.00:
							limit = l1*average_weekly_traffic
						elif (net_revenue >= 50.00) and (net_revenue < 100):
							limit = l2*average_weekly_traffic
						elif (net_revenue >= 20.00) and (net_revenue < 50):
							limit = l3*average_weekly_traffic
						else:
							limit=l4*average_weekly_traffic
							
						return limit


					lower_limit_fun_udf = udf(lower_limit_fun, DoubleType())


					region_df_algo_final= (region_df_algo_treated.withColumn("objective_fun",objective_fun_udf("net_revenue_new" ,"predicted_new"))
												.withColumn("Lower_Limit", lower_limit_fun_udf("net_revenue_new" , "AVERAGE_WEEKLY_TRAFFIC"))
												.withColumn("id", monotonicallyIncreasingId())
												.withColumn("var_name" ,var_name_udf("id"))
												.rdd
												.repartition(1)
												.cache())

					region_count = region_df_algo_final.count()


					##list of variuable names for opti algo
					t_list = region_df_algo_final.map(lambda x  : x.var_name).collect()

					##objective function for lp solve
					objective_list =  region_df_algo_final.map(lambda x  : x.objective_fun).collect()
					final_objective = {t_list[i]: objective_list[i] for i in range(0, region_count)}

					##lower and upper limit 
					lower_limit_list =  region_df_algo_final.map(lambda x  : x.Lower_Limit).collect()
					upper_limit_list =  region_df_algo_final.map(lambda x  : x.AVERAGE_WEEKLY_TRAFFIC).collect()
					lower_limit_final = {t_list[i]: lower_limit_list[i] for i in range(0, region_count)}
					upper_limit_final = {t_list[i]: upper_limit_list[i] for i in range(0, region_count)}


					## Optimization Algorithm
					cnt = 0.5
					lp_status = "Start"
					region_name = region_df_algo_final.first()[0]

					while ((lp_status != "Optimal")  & (cnt <=5.0)):
						a = cnt
						
						def constraint_fun(row):
							last_row = row.CPC_HCBA - a * row.objective_fun
							return last_row
						
						last_row = region_df_algo_final.map(lambda x: constraint_fun(x)).collect()
						
						final_constraint = {t_list[i]: last_row[i] for i in range(0, region_count)}
						
						prob = LpProblem("The Optimization Problem", LpMaximize)
						
						lp_var = {}
						for variable in t_list:
							variable = LpVariable(variable,lower_limit_final[variable],upper_limit_final[variable])
							lp_var[str(variable)] = variable
						
						prob += lpSum([final_objective[i]*lp_var[i] for i in t_list])
						
						prob += lpSum([final_constraint[i]*lp_var[i] for i in t_list]) == 0
						
						prob.solve()
						
						lp_status = LpStatus[prob.status]
						
						var_values = {"start":0}
						for v in prob.variables():
							var_values.update({v.name[2:]:v.varValue})
					   
						results={"region_name":region_name,"lp_status":LpStatus[prob.status],"cnt":(cnt), "gp":value(prob.objective),"var_list":var_values}
						
						cnt += 0.1
						
					final_results= [results]



					import pandas
					region_df_algo_final = sqlContext.createDataFrame(region_df_algo_final) #changing back to sqlcontext

					field = [StructField("var_id", StringType(), True),StructField("var_value", DoubleType(), True),StructField("Efficiency", FloatType(), True),StructField("Model", StringType(), True)]
					schema = StructType(field)

					optimization_df_algo1 = sqlContext.createDataFrame(sc.emptyRDD(), schema)

					for i in range(len(final_results)):
						optimization_df_algo1 = optimization_df_algo1.unionAll(df_create(final_results[i]))

					optimization_df_algo1 = optimization_df_algo1.cache()
					#final_results.unpersist()

					optimization_df_algo1 = optimization_df_algo1.withColumn("id",optimization_df_algo1["var_id"].cast(LongType()))
					optimization_df_algo2 = region_df_algo_final.select([col for col in region_df_algo_final.columns if col not in {"predicted_new","net_revenue_new","objective_fun","Lower_Limit","var_name","part_id"}]).join(optimization_df_algo1,"id","inner")



					optimization_df_algo3 = optimization_df_algo2.withColumn("threshold",threshold_udf("var_value","AVERAGE_WEEKLY_TRAFFIC")).                        withColumn("traffic_allow",traffic_allow_udf("var_value","AVERAGE_WEEKLY_TRAFFIC"))

					optimization_df_algo3 = optimization_df_algo3.select([col for col in optimization_df_algo3.columns if col not in {"id","var_id","Model"}])
					optimization_df_algo3 = optimization_df_algo3.withColumnRenamed("var_value","Lp_solution")
					optimization_df_algo_final = optimization_df_algo3.select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","threshold","EPB","traffic_allow")

					hcba_updatedcols = (hcba_opti_final.select('HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE','CPC_HCBA','AVERAGE_WEEKLY_TRAFFIC')
													.withColumnRenamed('AVERAGE_WEEKLY_TRAFFIC','AVERAGE_WEEKLY_TRAFFIC-usedintraining')
													.withColumnRenamed('CPC_HCBA','CPC_HCBA-usedintraining'))
													
					# the data is already in a spark dataframe, no need to read in as pandas and then convert
					# using validation data first
					# validation_df_spark = rf_validation_data
					validation_df_spark = validation_ad.join(hcba_updatedcols,['HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE'],'left_outer') #changing so that we have the predicted conversion rate as well
					validation_df_spark1=validation_df_spark.filter(validation_df_spark.DEVICE_TYPE=='DESKTOP')

					validation_df_spark2 = validation_df_spark1.withColumn("LOS_BUCKET_1",change_los_udf("LOS_BUCKET")).withColumn("BW_BUCKET_1",change_bw_udf("BW_BUCKET")).drop("LOS_BUCKET").drop("BW_BUCKET").withColumnRenamed("LOS_BUCKET_1","LOS_BUCKET").withColumnRenamed("BW_BUCKET_1","BW_BUCKET")

					validation_df_spark3 = validation_df_spark2.filter(validation_df_spark2.MODEL_NAME == name)

					common_queries = validation_df_spark3.join(optimization_df_algo_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")

					valid_df = common_queries.withColumn("throttle_traffic",(common_queries.traffic_allow*common_queries.TRAFFIC))

					valid_df = valid_df.withColumn("throttle_cost",((valid_df.COST*valid_df.throttle_traffic)/valid_df.TRAFFIC))
					valid_df = valid_df.withColumn("average_booking",(valid_df.BOOKING*valid_df.traffic_allow))
					valid_df = valid_df.withColumn("average_earning",((valid_df.GROSS_PROFIT*valid_df.average_booking)/valid_df.BOOKING))

					const = str(l1)+"_"+str(l2)+"_"+str(l3)+"_"+str(l4)

					if counter == 1: #change it to the first number from which starting to run
						valid_actual=valid_df.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"})
						#valid_actual.toPandas().to_csv("/home/neyre/valid_actual_CANADA_set3.csv")
						
						valid_throttle = (valid_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
											.withColumn("constraint",lit(const)))
						
					else :
						valid_throttle = valid_throttle.unionAll((valid_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
																 .withColumn("constraint",lit(const))))
					
					
					'''if counter in [100,200,500,700,1000]:
						valid_throttle.toPandas().to_csv("/home/neyre/valid_throttle_CANADA_set4_1_100.csv")''' ##change the file name
					

					# using training data second
					training_df_spark = training_ad.join(hcba_updatedcols,['HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE'],'left_outer')
					training_df_spark1= training_df_spark.filter(training_df_spark.DEVICE_TYPE=='DESKTOP').filter(training_df_spark.ACTIVITY_DATE>'2016-01-01')

					training_df_spark2 = training_df_spark1.withColumn("LOS_BUCKET_1",change_los_udf("LOS_BUCKET")).withColumn("BW_BUCKET_1",change_bw_udf("BW_BUCKET")).drop("LOS_BUCKET").drop("BW_BUCKET").withColumnRenamed("LOS_BUCKET_1","LOS_BUCKET").withColumnRenamed("BW_BUCKET_1","BW_BUCKET")

					training_df_spark3 = training_df_spark2.filter(training_df_spark2.MODEL_NAME == name)

					common_queries_train = training_df_spark3.join(optimization_df_algo_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")

					train_df = common_queries_train.withColumn("throttle_traffic",(common_queries_train.traffic_allow*common_queries_train.TRAFFIC))

					train_df = train_df.withColumn("throttle_cost",((train_df.COST*train_df.throttle_traffic)/train_df.TRAFFIC))
					train_df = train_df.withColumn("average_booking",(train_df.BOOKING*train_df.traffic_allow))
					train_df = train_df.withColumn("average_earning",((train_df.GROSS_PROFIT*train_df.average_booking)/train_df.BOOKING))

					
					
					if counter == 1:#change it to the first number from which starting to run
						train_actual = train_df.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"})
						#train_actual.toPandas.to_csv("/home/vireddy/train_actual_us_set3.csv")
						
						train_throttle = (train_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
											.withColumn("constraint",lit(const))) 
											
					else :
						train_throttle = train_throttle.unionAll((train_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
																.withColumn("constraint",lit(const))))
																
#					if counter in [100,200,500,700,1000]:
#						train_throttle.toPandas().to_csv("/home/vireddy/train_throttle_na_set3_1_100.csv") ##change the file name
                
				counter += 1
				print(counter)

valid_throttle.show(50)
valid_actual.show()'''